In [1]:
import os
import math
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import all_estimators
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.insert(0, "../scripts")

In [3]:
from decomposition import TimeSeriesDecomposer

In [4]:
DATA_PATH = '../data/raw/pinheiro'

In [5]:
df_sales = pd.read_csv(os.path.join(DATA_PATH, 'sales.csv'))

In [6]:
df_sales = df_sales.groupby(['date', 'product_code'])['unit_sales'].mean().reset_index()

In [7]:
df_sales = df_sales[df_sales['date'] < '2019-12-01']

In [8]:
product_avg_sales = df_sales.groupby(['product_code'])['unit_sales'].mean().reset_index().sort_values(by='unit_sales', ascending=False)

In [9]:
product_avg_sales

,product_code,unit_sales
192,104649,938.768914
126,58008,393.344220
140,60504,182.699131
110,52060,177.045465
10,1386,153.792687
...,...,...
228,126523,2.357143
6,1292,2.276400
177,88073,0.668233
240,131935,0.158365


In [10]:
df_product = pd.read_csv(os.path.join(DATA_PATH, 'products.csv'))

In [11]:
product_avg_sales.merge(df_product[['product_code', 'description']], on='product_code')

,product_code,unit_sales,description
0,104649,938.768914,CERV.SKOL 300ML RETORNAVEL (LIQUIDO)
1,58008,393.344220,LEITE PO ITAMBE INTEGR.200G SCH
2,60504,182.699131,ACHOC.LIQ.NESCAU PRONTINHO 200ML
3,52060,177.045465,CERV.SCHIN.350ML PILSEN LT
4,1386,153.792687,BATATA INGLESA KG
...,...,...,...
237,126523,2.357143,"GOMA MASC.TRIDENT SAB.18S MENTA 30,6G"
238,1292,2.276400,CHESTER PERDIGAO CONG.KG
239,88073,0.668233,LEITE L.VIDA BETANIA DESN.UHT VITAM.1L
240,131935,0.158365,BEB.LACT.ITAMBE GOODY MORANGO BDJ 540G


In [12]:
df = df_sales[df_sales['product_code'] == 104649]

In [13]:
df.rename(columns={
    'unit_sales': 'y'
}, inplace=True)

In [14]:
ts_decomposer = TimeSeriesDecomposer(df)

In [15]:
ts_decomposer.decompose(
    m=7, season=7, decomposition_type='classical', classical_method='additive'
)

,date,product_code,y,ma_7,trend,detrend_y,seasonality,remainder,at,st
192,2017-01-01,104649,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,2017-01-02,104649,219.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,2017-01-03,104649,663.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
918,2017-01-04,104649,428.250,776.535714,776.535714,-348.285714,NaN,NaN,NaN,NaN
1160,2017-01-05,104649,1201.375,879.017857,879.017857,322.357143,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
256470,2019-11-26,104649,1091.250,1096.214286,1096.214286,-4.964286,27.377551,-32.341837,1063.872449,27.377551
256712,2019-11-27,104649,897.125,942.875000,942.875000,-45.750000,79.181122,-124.931122,817.943878,79.181122
256954,2019-11-28,104649,944.625,1037.458333,1037.458333,-92.833333,14.671769,-107.505102,929.953231,14.671769
257196,2019-11-29,104649,1340.375,1133.200000,1133.200000,207.175000,35.275850,171.899150,1305.099150,35.275850


In [19]:
x = datetime(2019, 11, 30, 0, 0, 0)

In [20]:
type(x)

datetime.datetime

In [21]:
a = True

In [22]:
type(a)

bool

In [12]:
def compute_ma(df, m):
    k = int((m-1)/2)
    ma = []
    for i in range(len(df)):
        i_start = i - k
        i_end = i + k + 1
        if i_start < 0:
            ma.append(np.nan)
        else:
            ma.append(df.iloc[i_start:i_end]['y'].mean())
            
    df[f'ma_{m}'] = ma
    return df


def decompose_ts(df, m, season):
    df = compute_ma(df, m)
    df['trend'] = df['ma_7']
    df['detrend_y'] = df['y'] - df['trend']
    df['seasonality'] = df['detrend_y'].rolling(window=season).mean()
    df['remainder'] = df['y'] - df['trend'] - df['seasonality']
    df['at'] = df['trend'] + df['remainder']
    df['st'] = df['seasonality']
    
    return df

class FeatureExtractor:
    def __init__(self, df, start_date):
        self.df = df
        self.date_list = df[df['date'] >= start_date]['date'].tolist()
        self.features = {'date': self.date_list}

    @staticmethod
    def get_previous_y(df, n, by_weekday, **kwargs):
        if by_weekday:
            date = kwargs.get('date')
            wd = date.weekday()
            df_weekday = df[df['weekday'] == wd]
            return df_weekday.iloc[-n:].sort_values(by='date', ascending=False)['y'].T.values
        else:
            return df.iloc[-n:].sort_values(by='date', ascending=False)['y'].T.values   

    @staticmethod
    def get_wom(date):
        wom = math.ceil(date.day/7)
        if wom > 4:
            return 4
        else:
            return wom
 
    def get_autoregressive_features(self, n_days, n_weeks):
        cols1 = [f'yt-{k}' for k in range(1, n_days+1)]
        cols2 = [f'yt-{k*7}' for k in range(1, n_weeks+1)]
        cols = cols1 + cols2
        autoregressive_data = {}
        for date in self.date_list:
            df_truncate = self.df[self.df['date'] < date].sort_values(by='date')
            values_daily = self.get_previous_y(df_truncate, n_days, False)
            values_weekly = self.get_previous_y(df_truncate, n_weeks, True, date=date)
            values = np.concatenate((values_daily, values_weekly))
            autoregressive_data[date] = values
            
        autoregressive_features = pd.DataFrame(autoregressive_data.values(), columns=cols)
        self.features['autoregressive'] = autoregressive_features

        
    def get_categorical_features(self):
        df_date = pd.DataFrame({'date': self.date_list})
        df_date = df_date.merge(self.df, on='date')[['date', 'weekday']]
        df_date['wom'] = df_date['date'].apply(lambda x: self.get_wom(x))
        df_date['month'] = pd.DatetimeIndex(df_date['date']).month
        df_date = df_date[df_date.columns[1:]]
        
        features = df_date.columns
        categorical_features = pd.DataFrame()
        for feature in features:
            df_dummy = pd.get_dummies(df_date[feature])
            df_dummy = df_dummy.rename(columns={
                col: feature + '_' + str(col) for col in df_dummy.columns
            })
            categorical_features = pd.concat([categorical_features, df_dummy], axis=1)

        self.features['categorical'] = categorical_features
        
    def extract(self, n_days, n_weeks):
        self.get_autoregressive_features(n_days, n_weeks)
        self.get_categorical_features()
        

def get_samples(features, df_, train_size):
    sample_keys = [
        ('autoregressive', 'at'),
        ('categorical', 'st')
    ]
    
    samples = {}
    for sample_key in sample_keys:
        feature_name, y_true_name = sample_key
        feature_samples = {}
        data = {
            'train': {
                'X': features[feature_name][:train_size],
                'y': df_[[y_true_name]][:train_size]
            },
            'test': {
                'X': features[feature_name][train_size:],
                'y': df_[[y_true_name]][train_size:]
            }
        }

        for set_ in data.keys():
            feature_samples[set_] = {}
            for k, vec in data[set_].items():
                scaler = MinMaxScaler()
                vec_scaled = scaler.fit_transform(vec)
                feature_samples[set_][k] = {
                    'data': vec_scaled,
                    'scaler': scaler
                }

        samples[feature_name] = feature_samples
        
    return samples

def eval_model(model, samples):
    result = {}
    for feature_name, feature_samples in samples.items():
        train_data = feature_samples['train']
        X_train, y_train = train_data['X']['data'], train_data['y']['data']

        test_data = feature_samples['test']
        X_test, y_test = test_data['X']['data'], test_data['y']['data']

        model.fit(X_train, y_train)

        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        train_scaler = train_data['y']['scaler']
        test_scaler = test_data['y']['scaler']

        y_true_train = train_scaler.inverse_transform(y_train)
        y_true_test = train_scaler.inverse_transform(y_test)

        y_pred_train = train_scaler.inverse_transform(y_pred_train.reshape(-1, 1))
        y_pred_test = train_scaler.inverse_transform(y_pred_test.reshape(-1, 1))

        result[feature_name] = {
            'train': {
                'true': y_true_train,
                'pred': y_pred_train
            },
            'test': {
                'true': y_true_test,
                'pred': y_pred_test
            }
        }

    y_true_train = result['autoregressive']['train']['true'] + result['categorical']['train']['true']
    y_pred_train = result['autoregressive']['train']['pred'] + result['categorical']['train']['pred']

    y_true_test = result['autoregressive']['test']['true'] + result['categorical']['test']['true']
    y_pred_test = result['autoregressive']['test']['pred'] + result['categorical']['test']['pred']

    results_metrics = {
        'mae/avg': {
            'train': mean_absolute_error(y_true_train, y_pred_train)/np.mean(y_true_train),
            'test': mean_absolute_error(y_true_test, y_pred_test)/np.mean(y_true_test)
        }
    }
    
    return results_metrics

In [21]:
# for model in MODEL_DICT.values():
#     print(eval_model(model, samples))

In [22]:
# fig = plt.figure(figsize=[16, 8])
# plt.plot(y_true_train[-100:], label='y_true')
# plt.plot(y_pred_train[-100:], label='y_pred')
# plt.legend()
# plt.show()

---

## Models

In [24]:
estimators = all_estimators(type_filter='regressor')

all_regs = {}
for name, RegressorClass in estimators:
    try:
        print('Appending', name)
        reg = RegressorClass()
        all_regs[name] = reg
    except Exception as e:
        print('--------------')
        print(name)
        print(e)
        print('--------------')
        
        
performances = {}
for name, model in all_regs.items():
    try:
        r = eval_model(model, samples)
        test_performance = r['mae/avg']['test']
        performances[name] = test_performance
    except Exception as e:
        print('--------------')
        print(name)
        print(e)
        print('--------------')

Appending ARDRegression
Appending AdaBoostRegressor
Appending BaggingRegressor
Appending BayesianRidge
Appending CCA
Appending DecisionTreeRegressor
Appending DummyRegressor
Appending ElasticNet
Appending ElasticNetCV
Appending ExtraTreeRegressor
Appending ExtraTreesRegressor
Appending GammaRegressor
Appending GaussianProcessRegressor
Appending GradientBoostingRegressor
Appending HistGradientBoostingRegressor
Appending HuberRegressor
Appending IsotonicRegression
Appending KNeighborsRegressor
Appending KernelRidge
Appending Lars
Appending LarsCV
Appending Lasso
Appending LassoCV
Appending LassoLars
Appending LassoLarsCV
Appending LassoLarsIC
Appending LinearRegression
Appending LinearSVR
Appending MLPRegressor
Appending MultiOutputRegressor
--------------
MultiOutputRegressor
__init__() missing 1 required positional argument: 'estimator'
--------------
Appending MultiTaskElasticNet
Appending MultiTaskElasticNetCV
Appending MultiTaskLasso
Appending MultiTaskLassoCV
Appending NuSVR
Append

---

## Analyse difficulty

In [53]:
trends = []
seasonalities = []
remainders = []
baseline_values = []

for product_code in tqdm(df_sales['product_code'].unique(), bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}'):
    try:
        df = df_sales[df_sales['product_code'] == product_code]

        df.rename(columns={
            'unit_sales': 'y'
        }, inplace=True)

        df['date']= pd.to_datetime(df['date'])
        df['weekday'] = df['date'].apply(lambda x: x.weekday())
        df['baseline'] = df['y'].rolling(window=7).mean()

        df = decompose_ts(df, 7, 7)
        start_date = datetime(2018, 1, 1)

        date_list = df[df['date'] >= start_date]['date'].tolist()

        train_ratio = 0.8
        df_ = df[df['date'].isin(date_list)]
        train_size = int(train_ratio * len(df_))

        y_pred_baseline = df_['baseline'].values.reshape(-1, 1)
        y_true = df_['y'].values.reshape(-1, 1)
        baseline_value = mean_absolute_error(y_true, y_pred_baseline)/np.mean(y_true)

        a = df[['trend', 'seasonality', 'remainder']].mean()
        trends.append(a.trend)
        seasonalities.append(a.seasonality)
        remainders.append(a.remainder)
        baseline_values.append(baseline_value)
        
    except:
        print(f"Error on product {product_code}")

100%|██████████████████████████████| 242/242 [00:32<00:00,  7.52it/s]


In [54]:
table = pd.DataFrame({
    'product_code': df_sales['product_code'].unique(),
    'trend': trends,
    'seasonality': seasonalities,
    'remainder': remainders,
    'baseline_value': baseline_values
})

In [55]:
table.sort_values(by='baseline_value')

,product_code,trend,seasonality,remainder,baseline_value
54,3017,28.654651,-0.001083,0.005092,0.079811
55,3260,9.010249,-0.001131,0.001437,0.104733
43,1655,76.412537,0.004289,-0.009316,0.107896
96,17183,67.763284,0.003438,0.003226,0.126692
93,10315,59.836242,0.002214,0.002642,0.127451
...,...,...,...,...,...
66,4121,4.577361,-0.000354,0.000541,1.013166
70,5490,4.783399,0.001585,-0.009259,1.037258
122,57729,34.194931,-0.003339,0.006910,1.041150
134,58556,30.722343,0.000813,-0.004515,1.117394


In [56]:
from scipy.stats import pearsonr

In [57]:
corr, _ = pearsonr(table['trend'], table['baseline_value'])
corr

0.09003615506586217

In [58]:
corr, _ = pearsonr(table['seasonality'], table['baseline_value'])
corr

-0.1697842451872687

In [59]:
corr, _ = pearsonr(table['remainder'], table['baseline_value'])
corr

0.18814399845484883